# Toronto Neighborhoods

Importing and installing useful libraries.

In [1]:
! pip install lxml html5lib beautifulsoup4
import pandas as pd
import numpy as np

     |████████████████████████████████| 5.5MB 837kB/s eta 0:00:01     |███████▌                        | 1.3MB 2.6MB/s eta 0:00:02     |███████████████████▋            | 3.4MB 2.6MB/s eta 0:00:01
     |████████████████████████████████| 122kB 5.9MB/s eta 0:00:01


Accessing the web page and storing the table in a dataframe called "postal_code_raw".

In [2]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
df = pd.read_html(url)

postal_code_raw = df[0]
postal_code_raw.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Dropping 'Not assigned' Borough and assigning to new df 'postal_code'.

In [3]:
postal_code_raw['Borough'].replace('Not assigned', np.nan, inplace=True) #Replacing 'Not Assigned' in Borough column with NA

postal_code = postal_code_raw.dropna() #Dropping rows containing NA
postal_code.reset_index(drop=True, inplace=True)
postal_code.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Grouping and sorting by postal code.

In [14]:
postal_code.groupby(['Postal Code'])
postal_code.sort_values(['Postal Code'])
postal_code.reset_index(drop=True, inplace=True)
postal_code.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Replacing 'Not assigned' Neighbourhood with Borough's name.

In [5]:
for row in range(0,len(postal_code)):
    if postal_code.loc[row,'Neighbourhood'] == 'Not Assigned':
        postal_code.loc[[row],['Neighbourhood']] = postal_code.loc[[row],['Borough']] 
        print('Row with index {} has not assigned neighbouhood.'.format(row))

In [6]:
postal_code.shape

(103, 3)

Getting coordinates for each postal code.

In [22]:
spatial_data_list = pd.read_html('https://github.com/filippodercoli92/Coursera_Capstone/blob/main/Geospatial_Coordinates.csv')
spatial_data = spatial_data_list[0]
spatial_data.head()

,Unnamed: 0,Postal Code,Latitude,Longitude
0,NaN,M1B,43.806686,-79.194353
1,NaN,M1C,43.784535,-79.160497
2,NaN,M1E,43.763573,-79.188711
3,NaN,M1G,43.770992,-79.216917
4,NaN,M1H,43.773136,-79.239476


Cleaning dataframe

In [23]:
spatial_data.drop('Unnamed: 0', axis=1, inplace=True)
spatial_data.sort_values(['Postal Code'])
spatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Appending spatial coordinates to postal code df.

In [30]:
postal_code.merge(spatial_data, on='Postal Code')

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
